# PDF Loader

In [15]:
from langchain.document_loaders import PDFPlumberLoader
file_path='.\Article_93.pdf'
loader = PDFPlumberLoader(file_path)
documents = loader.load()



# Split, Vector Embedding and Store

In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [4]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [ ]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=100)
chunks = text_splitter.split_documents(documents) 

In [14]:
# 3. Try creating the vector database
import chromadb

vector_db = Chroma.from_documents(
    documents=chunks,
    persist_directory="./chroma_db", 
    embedding=OllamaEmbeddings(model="llama3"),
    collection_name="local-rag"
)

In [ ]:
#read vector from local database
vector_db = Chroma(
    persist_directory="./chroma_db",  
    embedding_function=OllamaEmbeddings(model="llama3"),  
    collection_name="local-rag"  
)

C:\Users\wento\AppData\Local\Temp\ipykernel_11784\1711387404.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(


## Retrieval

In [7]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Model Initialization

In [8]:
# LLM from Ollama
local_model = "llama3"
llm = ChatOllama(model=local_model)

In [9]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [10]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [11]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question

In [12]:
chain.invoke("What's the main focus of this pdf?")

'The main focus of this PDF is the application of machine learning and Apache Spark to analyze sales trends and make predictions in a retail setting, specifically Walmart stores.'

In [ ]:
# use this after each request to clear  cache
chromadb.api.client.SharedSystemClient.clear_system_cache()